## 카카오톡 파일 처리 함수

In [1]:
import os
os.chdir("C:/Users/beomj/Onedrive/바탕 화면");

        
def conversation_to_list(kakaotalk_file_name, Month, Day, AM, Hour, Minute):
    
    length_of_list = 20
    name_list = []
    talk_list = []
    
    f = open(kakaotalk_file_name,'rt',encoding='UTF8');
    lines = f.readlines()
    #check Month, Day, Hour, Minute and choose 20 words
    temp_lines = []
    m=0
    d=0
    c=0
    for line in lines:
        if(line.find(str(Month)+"월 "+str(Day)+"일") !=-1 and line.find('[')==-1):
            m=1
        if(m==1 and AM and line.find("오전 "+str(Hour)+":"+Minute)!=-1):
            d=1
        if(m==1 and not AM and line.find("오후 "+str(Hour)+":"+Minute)!=-1):
            d=1
        if(m==1 and d==1):
            temp_lines.append(line)
            c += 1
        if(c== length_of_list):
            break
    lines = temp_lines
    
    # get lines 
    for line in lines:    
        if(line.find('[')==0):
            index1 = line.find('[') +1
            index2 = line.find(']')
            index3 = line.find(']',index2+1) +1

            name2 = line[index1:index2]
            name_list.append(name2)
            talk_list.append(line[index3+1:])

    f.close()

    return name_list, talk_list


## 파파고 번역 함수

In [2]:
import requests
# 파파고 스트링 번역

def papago_translate(context):
    try:
        USER_ID = "svrfjDI6SLeRFekWQEOz" # 개발자센터에서 발급받은 Client ID 값
        USER_SECRET = "JLKglR7QQM" # 개발자센터에서 발급받은 Client Secret 값
        url = "https://openapi.naver.com/v1/papago/n2mt"
        headers= {"X-Naver-Client-Id": USER_ID, "X-Naver-Client-Secret":USER_SECRET}
        params = {"source": "ko", "target": "en", "text": context}
        response = requests.post(url, headers=headers, data=params)
        res = response.json()
        return res['message']['result']['translatedText']
    except:
        return "PAPAGO_FAILED"

# talk_list 번역
def conv_translate(talk_list):
    translated_list = []
    for i in talk_list:
        new = papago_translate(i)
        translated_list.append(new)
    return translated_list


## 감정 계산 함수

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


def make_sent_list(trans_list):
    sent_list = []
    senti_analyzer = SentimentIntensityAnalyzer()
    for i in trans_list:
        score=senti_analyzer.polarity_scores(i)['compound']
        sent_list.append(score)
    return sent_list
    
    
def chat_labeling(score):
    if score>=0.3:
        return 'Good'
    elif score<=-0.3:
        return 'Bad'
    else:
        return "SoSo"
